In [1]:
# Some code is shamelessly copied from Learning Blogs
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame
from pandas import concat
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from pylab import rcParams
import math
import zipfile
from datetime import datetime
from dateutil import parser
import datetime
import random
from functools import reduce
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor


pd.set_option('display.max_columns', None)
rcParams['figure.figsize'] = 15, 5

def gt(x,y): 
    if x > y:
        return 1
    else:
        return 0
    
def le(x,y): 
    if x <= y:
        return 1
    else:
        return 0


def parseDate(d):
    try:
        return parser.parse(d)        
    except:
        return None

def maxD(v):
    dt1 = parseDate(v[0])
    dt2 = parseDate(v[1])
    if dt1 is not None and dt2 is not None:
        return max(dt1, dt2)
    elif dt1 is not None:
        return dt1
    else:
        return dt2
    
def addDelta(v):
    delta = datetime.timedelta(0,random.randint(1, 15), random.randint(1, 1001))
    dt = parseDate(str(v[0]))
    if dt is not None:
        return dt+delta
    else:
        return dt
    

    
def num(s):
    try:
        return int(s[0])
    except ValueError:
        return float(s[0])
    except ValueError:
        return None

def differ(v):
    d1 = num(v[0])
    d2 = num(v[1])
    if d1 is not None and d2 is not None and abs(d1-d2)<=10:
        return False
    else:
        return True
    
def pererr(err, avg):
    return (err/avg)*100

def mean(l):
    return reduce(lambda x, y: x + y, l) / len(l)

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)


# invert differenced value
def inverse_difference(history, pred, interval=1):
    return pred + history[-interval]

    
def predict(coef, history):
    pred = 0.0
    for i in range(1, len(coef)+1):
        pred += coef[i-1] * history[-i]
    return pred    

def calculateErrs(test, pred):
    errs = []
    for i in range(len(test)):
        err=pererr(abs(test[i]-pred[i]), test[i])
        errs.append(err)
    rmse = sqrt(mean_squared_error(test, pred))
    mae = mean_absolute_error(test, pred)
    print('Test RMSE: %.3f, MAE:%.3f, Err:%.3f'% (rmse, mae, mean(errs)))
    return errs
    
    
def errorRolls(errs):    
    counters = [np.array([le(x,2), le(x,5), le(x,7), le(x,10), le(x,15), gt(x,15)]) for x in errs]
    total = len(counters)*1.0
    errorRollUp = zip([2, 3, 5, 7, 10, 15, 15], reduce((lambda x,y : x+y), counters))
    return [(x[0], ((x[1]/total)*100)) for x in errorRollUp] 

## CLUB DataFrame Based on TruckTypes
def clubTruckTypes(truck_types_to_club, kpi_groupBy):
    all_groups=[]
    tobe_clubbed={}
    ## Group certain DF based on TruckTypes
    for (x, y) in kpi_groupBy:
        if x[3] in truck_types_to_club:
            key = (x[0], x[1], x[2])
            if tobe_clubbed.get(key):
                tobe_clubbed[key].append(y)
            else:
                tobe_clubbed[key] = [y]
        else:
            all_groups.append((x, y))            
    for (k,v) in tobe_clubbed.iteritems():
        key = k+(truck_types_to_club[0],)
        all_groups.append((key, pd.concat(v)))        
    return all_groups

## Convert DATAFrame to Series with Resampling & Forward/Backword Filling NANs
def getSeriesWithResampleAndFNA(kpi, groupBY, truck_types_to_club, ptr="ptr", date="d", W=8):
    kpi_ = kpi[(pd.isnull(kpi[ptr])==False)]
    kpi_groupBy = kpi_.groupby(groupBY)
    kpi_groupBy = [x for x in list(kpi_groupBy) if len(x[1]) >= W]
    if truck_types_to_club:
        kpi_groupBy =  clubTruckTypes(truck_types_to_club, kpi_groupBy)   
    all_series = [(x[0], Series(x[1][ptr].values, index=x[1][date].values)) for x in kpi_groupBy]
    all_series = [(x, s.sort_index()) for (x, s) in all_series]
    all_series = [(x, s.resample("1d").fillna(method="ffill").fillna(method="bfill")) for (x, s) in all_series]
    return [(x, s) for (x, s) in all_series if len(s) > W]


## Convert DATAFrame to Series without Any Resampling
def getSeriesWithoutResample(kpi, groupBY, truck_types_to_club, ptr="ptr", date="d", W=8):
    kpi_ = kpi[(pd.isnull(kpi[ptr])==False)]
    kpi_groupBy = kpi_.groupby(groupBY)
    if truck_types_to_club:
        kpi_groupBy =  clubTruckTypes(truck_types_to_club, kpi_groupBy)   
    kpi_groupBy = [x for x in list(kpi_groupBy) if len(x[1]) >= W]
    all_series = [(x[0], Series(x[1][ptr].values, index=x[1][date].values)) for x in kpi_groupBy]
    all_series = [(x, s.sort_index()) for (x, s) in all_series]
    return [(x, s) for (x, s) in all_series if len(s) >= W]

## get moving window features from the series for W size window with differencing
def getMovingWindowFeatures(series, W, last=False):
    X = series.values
    # converting time to epoch
    time = [t.to_datetime64().astype(np.int64)//10 ** 9 for t in series.index]
    # move each window(W+2) by 1  data point
    # keep last chunk for prediction
    NWs = len(X)-W-1 

    # Features should have dimensions (NWs x WSize)
    # Labels should have dimensions (NWs)
    feat = np.zeros( [NWs, 2*W] ) - 9999999.
    labl = np.zeros( [NWs] ) - 99999
    delta = [0]*NWs
    
    # Now fill the train feature and label arrays with differencing
    for i in range(NWs):
        feat[i,:] = np.concatenate((time[i:i+W] - time[i+W], X[i:i+W] - X[i+W]))        
        assert np.isfinite(feat[i, :]).all(), (W, len(X), NWs, i, series)
        labl[i]   = X[i+W+1] - X[i+W]    
        delta[i] = X[i+W]

    if last:
        n = len(X)-1
        return (feat, labl, delta, np.concatenate((time[n-W:n] - time[n], X[n-W:n] - X[n])), X[n])
    return (feat, labl, delta)

# get features for all the series with given window size
def getFeaturesForAllSeries(all_series, keys, W=7):
    all_series_features = [(x[0], getMovingWindowFeatures(x[1], W, True)) for x in all_series]
    train_feat, train_labl = [], []
    test_feat, test_delta = [], []
    for (x,(feat,lbl,_,pf,delta)) in all_series_features:
        for (f, l) in zip(feat, lbl):
            train_feat.append(dict(zip (keys, list(x)+f.tolist())))
            train_labl.append(l)
        test_feat.append(dict(zip (keys, list(x)+pf.tolist())))
        test_delta.append(delta)        
    return ((train_feat, train_labl), (test_feat, test_delta))

# get features for all the series with given window size and split into training and testing datasets
def getFeaturesForAllSeriesWithSplit(all_series, keys, W=7, train_frac = 0.5):
    all_series_features = [(x[0], getMovingWindowFeatures(x[1], W)) for x in all_series]    
    train_feat, train_labl = [], []
    test_feat, test_labl = [], []
    test_delta = []    
    for (x,(feat,lbl, delta)) in all_series_features:
        t_size = int(len(feat)*train_frac)    
        for (f, l) in zip(feat[:t_size], lbl[:t_size]):
            train_feat.append(dict(zip (keys, list(x)+f.tolist())))
            train_labl.append(l)
        for (f, l) in zip(feat[t_size:], lbl[t_size:]):
            test_feat.append(dict(zip (keys, list(x)+f.tolist())))
            test_labl.append(l)
        test_delta = test_delta + delta[t_size:]    
    return ((train_feat, train_labl), (test_feat, test_labl, test_delta))

# Transform Features into Vectorize form
def transformFeatures(train_feat, test_feat, vec = DictVectorizer()):
    train_feat_t = vec.fit_transform(train_feat).toarray()
    test_feat_t = vec.transform(test_feat).toarray()
    return (train_feat_t, test_feat_t)

def splitData(all_series_fna, keys, W=7, train_frac = 0.5):
    all_series_fna_featured = [(x[0], toWindowFeatures(x[1], 5)) for x in all_series_fna]    
    train_feat, train_labl = [], []
    test_feat, test_labl = [], []
    test_delta = []    
    for (x,(feat,lbl, delta)) in all_series_fna_featured:
        t_size = int(len(feat)*train_frac)    
        for (f, l) in zip(feat[:t_size], lbl[:t_size]):
            train_feat.append(dict(zip (keys, list(x)+f.tolist())))
            train_labl.append(l)
        for (f, l) in zip(feat[t_size:], lbl[t_size:]):
            test_feat.append(dict(zip (keys, list(x)+f.tolist())))
            test_labl.append(l)
        test_delta = test_delta + delta[t_size:]    
    return ((train_feat, train_labl), (test_feat, test_labl, test_delta))

# Run Improved RFR with given dataframe df, Window Size W and T as in # of predictions to do
def runIRFRForDF(df, groupBY, keys, truck_types_to_club,  W=7, T=1):
    rfr = RandomForestRegressor(n_estimators=100, 
                            criterion='mae', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, 
                            max_features='auto', max_leaf_nodes=None, 
                            bootstrap=True, oob_score=False, n_jobs=-1, 
                            random_state=None, verbose=0)
    all_series = getSeriesWithoutResample(df, groupBY, truck_types_to_club)
    ((train_feat, train_labl), (test_feat, delta)) = getFeaturesForAllSeries(all_series, keys)
    # Transform the features
    (train_feat_t, test_feat_t) = transformFeatures(train_feat, test_feat)
    # Learn IRFR
    rfr.fit(train_feat_t, train_labl)
    pred = rfr.predict(test_feat_t)    
    pred_act = []
    for i in range(len(pred)):
        pred_act.append(pred[i]+delta[i])
    return {(":".join([t[k] for k in groupBY])):(p, t) for t,p in zip(test_feat, pred_act)}        

def learnRFR(train_feat_t, train_labl):
    rfRegress = RandomForestRegressor(n_estimators=100, 
                            criterion='mae', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, 
                            max_features='auto', max_leaf_nodes=None, 
                            bootstrap=True, oob_score=False, n_jobs=-1, 
                            random_state=None, verbose=0)
    rfRegress.fit(train_feat_t, train_labl)
    return rfRegress

def checkNull(test_feat_t):
    return [e for e in test_feat_t if not np.isfinite(e).all()]


def IRFR(kpi):
    all_series = getSeriesWithResampleAndFNA(kpi)
    ((train_feat, train_labl), (test_feat, delta)) = makeFeaturesAndSplit(all_series)
    (train_feat_t, test_feat_t) = transformFeatures(train_feat, test_feat)
    rfr=learnRFR(train_feat_t, train_labl)
    pred = rfr.predict(test_feat_t)    
    pred_act = []

    for i in range(len(pred)):
        pred_act.append(pred[i]+delta[i])
    return (test_feat, pred_act)

/Library/Python/2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
kp = pd.read_csv("data/order-reports_09_10_2017_19_45.csv.zip", header=0)
kpi=kp.iloc[:, 
          [0, 2, 6, 7, 8, 9, 10, 11, 14, 20, 22, 
           37, 38, 39, 75, 41, 93, 95]]
# Picking earliest of Order Accepted or Order Blocked date as Order Date
kpi['date']=kpi[['Order Accepted', 'Order Blocked']].apply(maxD, axis=1)
#kpi.drop(['Order Accepted', 'Order Blocked'], axis=1, inplace=True)
## Checking how much bad Values/Rows we have
kpi[(pd.isnull(kpi['Freight Amount'])==False) & (pd.isnull(kpi['Order Value'])==False)].count()
# droping all rows where we don't know about date (i.e date is null)
kpi = kpi[pd.isnull(kpi['date'])==False]
# calculating per unit km per ton price for the lane
kpi['ptr'] = (kpi['Freight Amount']/kpi['Tonnage'])
# Lets try to make d as unique as possible by adding some secs, ms
kpi['d'] = kpi[['date']].apply(addDelta, axis=1)
kpi['d'].unique().size
kpi=kpi[kpi['d']>= parseDate("08/01/2017")]
kpi['epoch'] =  kpi['d'].astype(np.int64)//10 ** 9 

# should come around the count of rows we have
kpi.count()

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,33,68,69,92,94,99,106,110,114,119,124,129,130,132,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Python/2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Id                     5417
Customer Name          5417
From City              5417
To City                5417
Distance               5415
From Sublocation       5417
To Sublocation         5417
Order Value            4367
Used truck Type        4373
Warehouse Start Loc    5417
Warehouse End Loc      5417
Sector Name            5417
Tonnage                5408
Per Ton Rate           4308
Product Name           5417
Freight Amount         4367
Order Accepted         5350
Order Blocked            86
date                   5417
ptr                    4367
d                      5417
epoch                  5417
dtype: int64

In [3]:
PARAM_TUPS = [
    { "from_sub" : 'Kandla', "to_sub" : 'Pathankot', "truck_name" : '30 MT', "product_type" : 'Coal' },
    { "from_sub" : 'Kandla', "to_sub" : 'Dabok', "truck_name" : '30 MT', "product_type" : 'Coal' },
    { "from_sub" : 'Kandla', "to_sub" : 'Dholka', "truck_name" : '30 MT', "product_type" : 'Coal' },
    { "from_sub" : 'Kandla', "to_sub" : 'Modinagar', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Ghaziabad', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Ludhiana', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Muzaffarnagar', "truck_name" : '34T', "product_type" : 'Coal' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Karnal', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Agra', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Pinglaj', "truck_name" : '27T/28T', "product_type" : 'Sugar' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Rabariyawas', "truck_name" : '30 MT', "product_type" : 'Coal' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Muzaffarnagar', "truck_name" : '34T', "product_type" : 'Petroleum Coke' },
    { "from_sub" : 'Kandla', "to_sub" : 'Rabariyawas', "truck_name" : '34T', "product_type" : 'Coal' },        
    { "from_sub" : 'Kandla', "to_sub" : 'Madhopur', "truck_name" : '30 MT', "product_type" : 'Coal' },        
    { "from_sub" : 'Tuna', "to_sub" : 'Pathankot', "truck_name" : '30 MT', "product_type" : 'Coal' },        
    { "from_sub" : 'Gandhidham', "to_sub" : 'Panipat', "truck_name" : '27T/28T', "product_type" : 'Textiles' },        
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Dholpur House', "truck_name" : '27T/28T', "product_type" : 'Oil' },
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Ambala', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Ghaziabad', "truck_name" : '27T/28T', "product_type" : 'Oil' },        
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Kashmere Gate', "truck_name" : '27T/28T', "product_type" : 'Oil' },
    #########
    { "from_sub" : 'Kandla', "to_sub" : 'Pathankot', "truck_name" : '27T/28T', "product_type" : 'Coal' },
    { "from_sub" : 'Kandla', "to_sub" : 'Madhopur', "truck_name" : '27T/28T', "product_type" : 'Coal' },        
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Agra', "truck_name" : '27T/28T', "product_type" : 'Oil' },
    { "from_sub" : 'Lilashah Nagar', "to_sub" : 'Dholpur House', "truck_name" : '34T', "product_type" : 'Oil' },
    { "from_sub" : 'Anjar', "to_sub" : 'Agra', "truck_name" : '21T/20T', "product_type" : 'Oil' },
]

filterBY = ['From City', 'To City', 'Product Name', 'Used truck Type', 'Distance', 'epoch', 'ptr']
groupBY = ['From City', 'To City', 'Product Name', 'Used truck Type']
keys = ['From City', 'To City', 'Product Name', 'Used truck Type', 
        'd1', 'd2', 'd3',
        'pd1', 'pd2', 'pd3']
truck_types_to_club = ['27T/28T', '30 MT','34T']

In [56]:
mixpanel = pd.read_csv("data/mixpanle.csv", header=0, delimiter=",")


In [60]:
mixpanel["start_time"]=mixpanel["start_time"].astype(np.datetime64)


/Library/Python/2.7/site-packages/pandas/core/internals.py:3224: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


In [68]:
kpi = mixpanel[mixpanel["start_time"]>= parseDate("08/01/2017")]
kpi.count()

event                  155061
start_time             155061
user_id                155061
sp_type                 72765
customer_id             84069
order_request_id        95400
order_request_index     80599
requested_price         93430
payment_type            93430
load                    93430
load_booked             11997
load_unit               93430
product_type            93506
price_quoted            78730
is_exchange_price       40613
button_type             67789
page_type               67789
export_day             155061
dtype: int64

In [62]:
orders = list(kpi.groupby("order_request_id"))

In [66]:
for order, grp in orders:
    print order, len(grp)

16209.0 2
16224.0 1
16264.0 1
16265.0 1
17434.0 2
23206.0 14
23209.0 13
23263.0 1
23266.0 12
23267.0 10
23280.0 16
23281.0 70
23282.0 51
23283.0 58
23284.0 4
23287.0 23
23289.0 12
23290.0 9
23291.0 7
23292.0 10
23295.0 4
23296.0 29
23297.0 19
23298.0 40
23299.0 17
23300.0 45
23301.0 18
23302.0 34
23303.0 29
23304.0 11
23305.0 29
23306.0 17
23307.0 27
23308.0 28
23309.0 28
23310.0 21
23311.0 23
23312.0 19
23314.0 32
23315.0 34
23316.0 39
23317.0 16
23318.0 22
23319.0 25
23320.0 29
23321.0 60
23322.0 98
23323.0 32
23324.0 28
23325.0 30
23326.0 36
23327.0 2
23328.0 33
23329.0 33
23330.0 7
23331.0 35
23332.0 15
23333.0 16
23334.0 4
23335.0 15
23337.0 49
23338.0 19
23339.0 50
23340.0 21
23341.0 23
23342.0 27
23343.0 41
23344.0 21
23345.0 30
23346.0 46
23347.0 17
23348.0 21
23349.0 29
23350.0 25
23351.0 3
23352.0 12
23353.0 11
23354.0 10
23355.0 5
23357.0 4
23358.0 12
23360.0 8
23361.0 2
23362.0 4
23363.0 9
23365.0 15
23366.0 3
23367.0 16
23368.0 5
23370.0 14
23371.0 13
23372.0 21
23373.0 15

24200.0 12
24202.0 81
24204.0 112
24205.0 26
24206.0 45
24207.0 58
24208.0 89
24209.0 53
24210.0 18
24211.0 22
24212.0 33
24213.0 27
24214.0 21
24215.0 42
24216.0 32
24217.0 17
24219.0 16
24220.0 13
24221.0 1
24222.0 27
24223.0 73
24224.0 32
24226.0 30
24227.0 13
24228.0 17
24229.0 17
24230.0 12
24231.0 19
24232.0 16
24233.0 33
24234.0 6
24235.0 19
24236.0 1
24237.0 10
24238.0 5
24239.0 25
24240.0 31
24241.0 32
24242.0 13
24243.0 16
24244.0 9
24245.0 27
24246.0 15
24247.0 57
24248.0 9
24249.0 29
24250.0 20
24251.0 63
24252.0 32
24253.0 28
24254.0 61
24255.0 11
24256.0 53
24257.0 19
24258.0 22
24259.0 32
24260.0 13
24261.0 16
24262.0 15
24263.0 25
24264.0 26
24265.0 12
24266.0 18
24267.0 9
24268.0 8
24269.0 15
24270.0 26
24271.0 8
24272.0 1
24273.0 7
24274.0 9
24275.0 29
24276.0 8
24277.0 11
24278.0 28
24279.0 41
24280.0 1
24281.0 15
24282.0 69
24283.0 16
24284.0 18
24285.0 21
24286.0 39
24287.0 8
24288.0 18
24289.0 29
24290.0 53
24291.0 6
24292.0 23
24293.0 38
24294.0 17
24295.0 32
242

25763.0 51
25764.0 8
25765.0 20
25766.0 20
25767.0 25
25768.0 57
25769.0 28
25770.0 51
25771.0 20
25773.0 27
25774.0 22
25776.0 8
25777.0 27
25778.0 26
25779.0 9
25780.0 16
25781.0 25
25782.0 34
25783.0 10
25784.0 12
25785.0 13
25786.0 64
25787.0 2
25788.0 1
25789.0 33
25790.0 17
25791.0 14
25792.0 19
25793.0 24
25794.0 9
25796.0 9
25797.0 22
25798.0 19
25800.0 13
25801.0 1
25802.0 6
25803.0 2
25805.0 6
25806.0 111
25808.0 117
25809.0 12
25810.0 7
25811.0 8
25812.0 13
25813.0 6
25814.0 11
25815.0 21
25816.0 6
25817.0 9
25818.0 28
25819.0 16
25822.0 32
25826.0 1
25827.0 12
25828.0 10
25829.0 5
25830.0 10
25831.0 53
25833.0 3
25834.0 92
25835.0 3
25836.0 7
25837.0 30
25839.0 13
25840.0 22
25841.0 2
25842.0 8
25843.0 31
25844.0 9
25845.0 14
25846.0 39
25847.0 24
25848.0 77
25849.0 38
25850.0 53
25851.0 3
25852.0 26
25854.0 9
25855.0 9
25856.0 10
25857.0 18
25860.0 12
25861.0 27
25862.0 22
25863.0 14
25864.0 20
25865.0 30
25866.0 21
25867.0 28
25868.0 17
25869.0 48
25870.0 26
25871.0 23
25

27258.0 71
27259.0 26
27260.0 9
27263.0 94
27265.0 29
27266.0 13
27267.0 47
27268.0 25
27269.0 2
27270.0 92
27271.0 24
27272.0 11
27273.0 48
27274.0 12
27275.0 10
27276.0 24
27277.0 32
27278.0 11
27279.0 21
27280.0 51
27281.0 34
27282.0 81
27283.0 54
27284.0 70
27285.0 40
27286.0 16
27287.0 35
27288.0 17
27289.0 39
27290.0 17
27291.0 7
27292.0 3
27293.0 1
27294.0 2
27295.0 4
27296.0 8
27297.0 10
27298.0 32
27299.0 12
27300.0 30
27301.0 10
27302.0 5
27303.0 24
27304.0 46
27305.0 52
27306.0 55
27307.0 7
27308.0 47
27309.0 36
27310.0 53
27311.0 14
27312.0 11
27313.0 37
27314.0 24
27315.0 2
27316.0 20
27317.0 26
27318.0 25
27319.0 47
27320.0 32
27321.0 25
27322.0 38
27323.0 29
27324.0 18
27325.0 36
27327.0 14
27328.0 15
27329.0 26
27331.0 64
27332.0 9
27333.0 6
27334.0 62
27335.0 39
27336.0 71
27337.0 9
27338.0 9
27339.0 11
27340.0 23
27341.0 5
27344.0 42
27345.0 26
27346.0 4
27347.0 10
27348.0 11
27349.0 29
27350.0 16
27351.0 14
27353.0 33
27354.0 24
27355.0 26
27356.0 13
27357.0 13
27358

In [55]:
mixpanel

,event,start_time,user_id,sp_type,customer_id,order_request_id,order_request_index,requested_price,payment_type,load,load_booked,load_unit,product_type,price_quoted,is_exchange_price,button_type,page_type,export_day
0,ACCEPT CUSTOMER PRICE,None,9055,fleet_owner,12559.0,23295.0,NaN,1250.0,Advance,300.0,300.0,T,Petroleum Coke,NaN,NaN,NaN,NaN,2017-09-01
1,ACCEPT CUSTOMER PRICE,None,17221,NaN,10907.0,27803.0,NaN,1700.0,ToPay: Cash + Cheque,30.0,30.0,T,Coal,NaN,f,NaN,NaN,2017-10-11
2,ACCEPT CUSTOMER PRICE,None,5551,NaN,13135.0,23329.0,NaN,2400.0,ToPay: Cheque,30.0,29.0,T,Yellow Peas,NaN,NaN,NaN,NaN,2017-09-01
3,ACCEPT CUSTOMER PRICE,None,15251,fleet_owner,14388.0,27822.0,NaN,2750.0,ToPay: RTGS,90.0,60.0,T,Coal,NaN,t,NaN,NaN,2017-10-11
4,ACCEPT CUSTOMER PRICE,None,13230,fleet_owner,10825.0,23338.0,NaN,2350.0,ToPay: Cheque,150.0,72.0,T,Coal,NaN,NaN,NaN,NaN,2017-09-01
5,ACCEPT CUSTOMER PRICE,None,8066,NaN,10194.0,27773.0,NaN,2500.0,Advance,27.0,27.0,T,Oil,NaN,f,NaN,NaN,2017-10-11
6,ACCEPT CUSTOMER PRICE,None,5551,NaN,11941.0,23339.0,NaN,2400.0,ToPay: Cheque,30.0,30.0,T,Yellow Peas,NaN,NaN,NaN,NaN,2017-09-01
7,ACCEPT CUSTOMER PRICE,None,10279,fleet_owner,10195.0,27890.0,NaN,2540.0,Advance,108.0,27.0,T,Oil,NaN,t,NaN,NaN,2017-10-11
8,ACCEPT CUSTOMER PRICE,None,11587,fleet_owner,6867.0,23318.0,NaN,2400.0,ToPay: RTGS,180.0,180.0,T,Coal,NaN,NaN,NaN,NaN,2017-09-01
9,ACCEPT CUSTOMER PRICE,None,7389,NaN,13048.0,27896.0,NaN,3230.0,ToPay: RTGS,35.0,32.0,T,Petroleum Coke,NaN,f,NaN,NaN,2017-10-11


In [72]:
mixpanel.sort_values(["start_time"]).count()

event                  155061
start_time             155061
user_id                155061
sp_type                 72765
customer_id             84069
order_request_id        95400
order_request_index     80599
requested_price         93430
payment_type            93430
load                    93430
load_booked             11997
load_unit               93430
product_type            93506
price_quoted            78730
is_exchange_price       40613
button_type             67789
page_type               67789
export_day             155061
dtype: int64

In [95]:
kpi=kpi.sort_values("start_time")

In [96]:
ogrps = kpi[kpi['event']=="CHECK LANE CARD"].groupby("order_request_id")

In [97]:
for order, grp in list(ogrps)[:1]:
    print order, len(grp)

23206.0 10


In [98]:
list(ogrps)[:1][0]

(23206.0,
                  event          start_time  user_id      sp_type  customer_id  \
 69758  CHECK LANE CARD 2017-09-01 01:40:37    15082          NaN      10186.0   
 69778  CHECK LANE CARD 2017-09-01 02:16:44    10455          NaN      10186.0   
 67641  CHECK LANE CARD 2017-09-01 03:14:21    13063          NaN      10186.0   
 67865  CHECK LANE CARD 2017-09-01 04:24:13     7384          NaN      10186.0   
 67905  CHECK LANE CARD 2017-09-01 04:29:52     9612          NaN      10186.0   
 70894  CHECK LANE CARD 2017-09-01 05:36:23     3361  fleet_owner      10186.0   
 67631  CHECK LANE CARD 2017-09-01 06:04:42     3022          NaN      10186.0   
 69872  CHECK LANE CARD 2017-09-01 06:05:36     3022          NaN      10186.0   
 67691  CHECK LANE CARD 2017-09-01 06:09:56     3085          NaN      10186.0   
 67999  CHECK LANE CARD 2017-09-01 06:38:19     3085          NaN      10186.0   
 
        order_request_id  order_request_index  requested_price payment_type  \
 69758 